In [1]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
from ruamel.yaml import YAML
from sklearn.linear_model import LinearRegression

In [2]:
from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

In [3]:
pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}
yaml = YAML()

kB = 1000 * Boltzmann / electron_volt

In [4]:
fe_cif_filepath = "data/fe.cif"
fe_structure_2atom = from_file(structure_file=fe_cif_filepath)
rotate_cell_45_degrees = SupercellTransformation(
    scaling_matrix=[[1, 1, 0],
                    [1, -1, 0],
                    [0, 0, 1]],
)
fe_structure_4atom = rotate_cell_45_degrees.apply_transformation(fe_structure_2atom)
print(fe_structure_4atom)

Full Formula (Fe4)
Reduced Formula: Fe
abc   :   4.053843   4.053843   2.866500
angles:  90.000000  90.000000  90.000000
Sites (4)
  #  SP      a    b     c
---  ----  ---  ---  ----
  0  Fe    0    0     0
  1  Fe    0.5  0.5  -0
  2  Fe    0.5  0     0.5
  3  Fe    1    0.5   0.5


In [5]:
fe_magnetic_patterns = yaml.load(Path("data/fe_magnetic_patterns.yml"))
pd.DataFrame(fe_magnetic_patterns["4atoms"]).style \
    .set_table_styles(html_table_style) \
    .hide_index()

nm,f-type,a-type,g-type
0,1,1,1
0,1,1,-1
0,1,-1,1
0,1,-1,-1


In [6]:
fe_neighbor_data = count_neighbors(cell_structure=fe_structure_4atom, r=3)

In [7]:
fe_neighbor_data

NeighborData(data_frame=   i  j subspecies_i subspecies_j distance_bin                               n
0  0  0  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  2
1  0  1  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  8
2  0  2  Fe           Fe           (1.2412309099740464, 2.6744809099740463]  8
3  0  3  Fe           Fe           (1.2412309099740464, 2.6744809099740463]  8
4  1  1  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  2
5  1  2  Fe           Fe           (1.2412309099740464, 2.6744809099740463]  8
6  1  3  Fe           Fe           (1.2412309099740464, 2.6744809099740463]  8
7  2  2  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  2
8  2  3  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  8
9  3  3  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  2, bins_data_frame=                                         distance_bin                               \
(1.2

---

## Notes from meeting

Below is a more filled in version of the code we discussed during our meeting. This fills in the missing values (i.e., puts back pairs like (1, 0), (2, 1), and so on) and does the sort. I also show how you can get the neighbor counts for each site, and the neighbor counts grouped by index **`j`** and **`distance_bin`**.

In [8]:
a = fe_neighbor_data.data_frame \
    .merge(fe_neighbor_data.data_frame.rename(columns={"j": "i", "i": "j"}), how="outer") \
    .sort_values(["i","distance_bin", "j"]) \
    .loc[:, ["i", "j", "distance_bin"]] \
    .reset_index(drop=True)

neighbors_per_site = a.groupby(["i"]).count().reset_index().rename(columns={"j": "count"}).loc[:, "count"].values
sites_per_distance_group = a.groupby(["i", "distance_bin"]).count().reset_index().rename(columns={"j": "count"}).loc[:, "count"].values
neighbor_indices = a["j"].values

Here's what the sorted and filled in table looks like:

In [9]:
a

,i,j,distance_bin
0,0,2,"(1.2412309099740464, 2.6744809099740463]"
1,0,3,"(1.2412309099740464, 2.6744809099740463]"
2,0,0,"(2.6744809099740463, 3.0585190900259525]"
3,0,1,"(2.6744809099740463, 3.0585190900259525]"
4,1,2,"(1.2412309099740464, 2.6744809099740463]"
5,1,3,"(1.2412309099740464, 2.6744809099740463]"
6,1,0,"(2.6744809099740463, 3.0585190900259525]"
7,1,1,"(2.6744809099740463, 3.0585190900259525]"
8,2,0,"(1.2412309099740464, 2.6744809099740463]"
9,2,1,"(1.2412309099740464, 2.6744809099740463]"


Here's the vectors you would want to have for use in the Monte Carlo simulation.

In [10]:
print("Index j sorted over index i and distances =", neighbor_indices)
print("Number of neighbors for each site =", neighbors_per_site)
print("Number of neighbors grouped over index i and distances =", sites_per_distance_group)

Index j sorted over index i and distances = [2 3 0 1 2 3 0 1 0 1 2 3 0 1 2 3]
Number of neighbors for each site = [4 4 4 4]
Number of neighbors grouped over index i and distances = [2 2 2 2 2 2 2 2]


End of notes from meeting

---

In [11]:
x = (np.array(fe_neighbor_data[1]))
nn_distances = (x[:,1]).astype(float)
nn_distances

array([2.48246182, 2.8665    ])

In [ ]:
n_sites = int(np.size(fe_structure_4atom))
first_nn_list = np.zeros(n_sites)
first_nn_list = [ (fe_structure_4atom.get_neighbors(fe_structure_4atom[i], r=nn_distances[0])) \
           for i in range(n_sites) ]
first_nn_list[6]

In [ ]:
list_fe_structure_4atom = list(fe_structure_4atom)
list_fe_structure_4atom[0]
#first_nn_list[6][0]

In [ ]:
type(list(fe_structure_4atom)), type(first_nn_list)

In [ ]:
nn_site =np.array( [ [i for i, j in enumerate(list(fe_structure_4atom)) \
             if j == first_nn_list[n] ] for n in range(n_sites)  ] )

In [ ]:
first_nn_list[6]